## Drill Hole Data Processing

Drilling hole is a traditional method to explore minerals or other resources (for example, fresh water.) By drilling holes, observing the rock samples from the drill holes and taking other physical or chemical methods to measure these samples could get a comprehensive understanding of the area where the drillhole locates from the perspective of geology, geochemistry, petrology, biostratigraphy, et. The features from all these aspects construct a good dataset for predict the existence of cores. 

As drilling-hole provides rock sample for other ways, we start our data preparation from it. 


In [ ]:
# import required packages
import pandas as pd
import os
import pickle
import sys
import matplotlib.pyplot as plt
import ipywidgets as widgets
pd.options.display.width=None
pd.options.display.max_columns=None


if sys.version_info >= (3, 6):
    from zipfile import ZipFile as zipfile
else:
    import zipfile36 as zipfile
    
url = "https://unearthed-exploresa.s3-ap-southeast-2.amazonaws.com/Unearthed_5_SARIG_Data_Package.zip" 
# enter the directory to save data
data_loc = './data'
file_name = 'Unearthed_5_SARIG_Data_Package.zip'

if os.path.isfile(os.path.join(data_loc, file_name)):
    print ("File exist")
    pass
else:
    # open and save the zip file onto computer
    url = urlopen(URL)
    output = open('./data/Unearthed_5_SARIG_Data_Package.zip', 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()
    

# list all the files in the dataset and group these data tables.     
files_in_dataset = []
file_name = 'Unearthed_5_SARIG_Data_Package.zip'
for file in zipfile(os.path.join(data_loc, file_name),'r').filelist:
    files_in_dataset.append(file.filename)
    
files_in_dataset



For this part of data cleaning, we will only use the following relevant files: 
 - 'SARIG_Data_Package/sarig_dh_core_exp.csv',
 - 'SARIG_Data_Package/sarig_dh_details_exp.csv',
 - 'SARIG_Data_Package/sarig_dh_litho_exp.csv',
 - 'SARIG_Data_Package/sarig_dh_petrophys_exp.csv',
 - 'SARIG_Data_Package/sarig_dh_reference_exp.csv',
 - 'SARIG_Data_Package/sarig_dh_strat_exp.csv'

### DRILL HOLE BASIC INFORMATION

In [ ]:
sarig_dh_reference_exp = pd.read_csv(
    zipfile(os.path.join(data_loc, file_name),'r').open('SARIG_Data_Package/sarig_dh_reference_exp.csv','r'), 
    sep=',', encoding='latin1')
sarig_dh_reference_exp['SAMREF_CNO'] = sarig_dh_reference_exp['SAMREF_CNO'].astype('Int64')
sarig_dh_reference_exp['PUBLICATION_DATE'] = pd.to_datetime(sarig_dh_reference_exp['PUBLICATION_DATE'])
sarig_dh_reference_exp.head(5)

In [ ]:
del sarig_dh_reference_exp

### DRILL HOLE DETAILED INFORMATION
This summarize what information will be included into the other data tables.

In [ ]:
sarig_dh_details_exp = pd.read_csv(
    zipfile(os.path.join(data_loc, file_name),'r').open('SARIG_Data_Package/sarig_dh_details_exp.csv','r'), 
    sep=',', encoding='latin1')
# sarig_dh_details_exp['SAMREF_CNO'] = sarig_dh_reference_exp['SAMREF_CNO'].astype('Int64')
sarig_dh_details_exp['MAX_DRILLED_DEPTH_DATE'] = pd.to_datetime(sarig_dh_details_exp['MAX_DRILLED_DEPTH_DATE'])
sarig_dh_details_exp.head(5)

The MINERAL_CLASS is of interest in our context, so it is necessary to extract the information on the drill holes with MINERAL_CLASS is labeled as 'Y'. Also, we are only interested in the drill holes in South Australia, so we limit our consideration in a zone Latitude [-38, -25] Longitude [128.9, 141]. 

In [ ]:
sarig_dh_details_exp[sarig_dh_details_exp['MINERAL_CLASS']=='Y'].head(6)

In [ ]:
# the interesting selection is the the ones in SA and mineral class
dh_details_mineral = sarig_dh_details_exp[
    sarig_dh_details_exp['LATITUDE_GDA2020'].between(-38.0, -25.0) &
    sarig_dh_details_exp['LONGITUDE_GDA2020'].between(128.9, 141.0) & 
    (sarig_dh_details_exp['MINERAL_CLASS']=='Y')
][
    ['DRILLHOLE_NO', 'DH_UNIT_NO', 'MAX_DRILLED_DEPTH','TARGET_COMMODITIES', 
     'SITE_NO', 'EASTING_GDA2020', 'NORTHING_GDA2020', 'ZONE_GDA2020',
     'LONGITUDE_GDA2020', 'LATITUDE_GDA2020', 'LONGITUDE_GDA94', 
     'LATITUDE_GDA94', 'ELEVATION_M', 'SURVEY_METHOD_CODE','SURVEY_METHOD']
]
dh_details_mineral.sample(5)

The identifiers for the drill holes. 

In [ ]:
dh_mineral_identifiers = dh_details_mineral[
    ['DRILLHOLE_NO','SITE_NO']
].drop_duplicates()
# dh_mineral_identifiers.sample(5)

In [ ]:
dh_mineral_identifiers.count()

Since "DRILLHOLE_NO" and "SITE_NO" have the same counts, they can be used exchanably as identifiers in the following.

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 8))
# ax = plt.plot(sarig_dh_details_exp[
#     sarig_dh_details_exp['MAX_DRILLED_DEPTH_DATE'].notnull()][
#     ['DRILLHOLE_NO', 'MAX_DRILLED_DEPTH_DATE']].drop_duplicates() [
#     'MAX_DRILLED_DEPTH_DATE'].apply(lambda x: x.year).value_counts().sort_index())
# plt.xlim(1970, 2020)
# plt.title("Drill Holes VS Years")
# plt.show()

This figure shows the number of drill holes for each year it reached its maximum drill depth. In the late 1990s and the decade after 2000, there were more drilling holes. 

In [ ]:
interested_dh_details_mineral = dh_details_mineral[
    ['DRILLHOLE_NO', 'DH_UNIT_NO', 'MAX_DRILLED_DEPTH','TARGET_COMMODITIES', 
     'SITE_NO', 'LONGITUDE_GDA2020', 'LATITUDE_GDA2020', 'ELEVATION_M', 
     'SURVEY_METHOD_CODE']
].drop_duplicates()

In [ ]:
del sarig_dh_details_exp

### DRILL HOLE CORE INFORMATION

This datatable provides metadata what data/sample were collected at drill holes. 

In [ ]:
sarig_dh_core_exp = pd.read_csv(
    zipfile(os.path.join(data_loc, file_name),'r').open('SARIG_Data_Package/sarig_dh_core_exp.csv','r'), 
    sep=',', encoding='latin1')
# sarig_dh_details_exp['SAMREF_CNO'] = sarig_dh_reference_exp['SAMREF_CNO'].astype('Int64')
#sarig_dh_core_exp['MAX_DRILLED_DEPTH_DATE'] = pd.to_datetime(sarig_dh_core_exp['MAX_DRILLED_DEPTH_DATE'])
sarig_dh_core_exp.sample(5)

In [ ]:
sarig_dh_core_exp.info()

In [ ]:
sarig_dh_core_exp.columns

The 'DRILLHOLE_NO' and 'SITE_NO' are used to identify the drillholes. The 'TRAY_NO' corresponds to 'DEPTH_FROM_M' and 'DEPTH_TO_M', and the 'ROCK_SAMPLE', 'ROCK_SAMPLE_LIBRARY', 'GEOCHEMISTRY', 'PETROLOGY', 'BIOSTRATIGRAPHY' are all related to the samples with this 'TRAY_NO'. Here, we can tell from the 'Y' or 'N' values of these columns that the sarig_dh_core_exp is a profile of the samples and their following studies. Obviously, we care more about the test results themselves than the indicator of existing such an investigation or not. So, here, we neglect these columns. 

The columns "\*\_FEET" contain the same information as those "\*\_M", so they will be dropped. Some other columns such as DRILLHOLE_NAME','LONGITUDE_GDA2020', etc.,  which either contain information not useful for prediction or actually duplicate the info of another columns, will be dropped too.

In [ ]:
interested_dh_core_exp = interested_dh_details_mineral.merge(
    sarig_dh_core_exp[
    ['DRILLHOLE_NO', 'TRAY_NO', 'DEPTH_FROM_M', 'DEPTH_TO_M', 'CORE_TYPE',  
     'SITE_NO']], 
    how='inner', 
    on=['DRILLHOLE_NO', 'SITE_NO'], 
    suffixes=('_detail', '_core')).drop_duplicates()
interested_dh_core_exp.sample(5)

In [ ]:
del sarig_dh_core_exp
del interested_dh_details_mineral

### DRILL HOLE LITHOLOGY

In [ ]:
sarig_dh_litho_exp = pd.read_csv(
    zipfile(os.path.join(data_loc, file_name),'r').open('SARIG_Data_Package/sarig_dh_litho_exp.csv','r'), 
    sep=',', encoding='latin1')
# sarig_dh_litho_exp['SAMREF_CNO'] = sarig_dh_litho_exp['SAMREF_CNO'].astype('Int64')
sarig_dh_litho_exp['LOGGING_DATE'] = pd.to_datetime(sarig_dh_litho_exp['LOGGING_DATE'])
sarig_dh_litho_exp.head(5)

In [ ]:
interested_dh_core_exp.dtypes

In [ ]:
sarig_dh_litho_exp.dtypes

In [ ]:
sarig_dh_litho_exp['LOG_NUMBER'].astype('Int64')

After dropping the columns with the same info, the lithology information of each depth stratum is available: major lithology and minor lithology. Lithology type, especially the order of lithology at different depth stratum could have relationship with core occurrence. 

The DEPTH_FROM_M and DEPTH_TO_M define the strata, suggest a kind of order and indicate the depth value. Also, these depths are related to the samples sent for other studies, such as lithology, geophysics, etc.  These depth are important features from which some other features can be extracted during later feature engineering. 

Since "interested_dh_core_exp" and "sarig_dh_litho_exp" both have the columns named as 'DEPTH_FROM_M', 'DEPTH_TO_M', it is necessary to explore the possibility of merge them. 

In [ ]:
interested_dh_litho_exp = interested_dh_core_exp.merge(
    sarig_dh_litho_exp[
    ['DRILLHOLE_NO','LOG_NUMBER', 'DEPTH_FROM_M', 'DEPTH_TO_M', 
     'MAJOR_LITHOLOGY_CODE', 'MINOR_LITHOLOGY_CODE', 'SITE_NO']],
    how='left', 
    on=['DRILLHOLE_NO', 'SITE_NO'], 
    suffixes=('_core', '_lith')).duplicates()

interested_dh_litho_exp['LOG_NO'] = interested_dh_litho_exp['LOG_NUMBER'].astype('Int64')
interested_dh_litho_exp.drop('LOG_NUMBER', axis=1, inplace=True)
interested_dh_litho_exp.sample(5)

__Let us look at the columns 'DEPTH_FROM_M', 'DEPTH_TO_M' from both  "interested_dh_core_exp" and "sarig_dh_litho_exp".__

In [ ]:
interested_dh_litho_exp[
    ['DRILLHOLE_NO','SITE_NO', 'LOG_NO', 'DEPTH_FROM_M_core',
     'DEPTH_TO_M_core', 'DEPTH_FROM_M_lith', 'DEPTH_TO_M_lith']
].set_index(['DRILLHOLE_NO','SITE_NO']).dropna()

From the above data frame, it is shown that the columns 'DEPTH_FROM_M', 'DEPTH_TO_M' from the two sources cannot be merged meaningfully. This means that the samples for the study of lithology might be taken at different depth. 

In [ ]:
del sarig_dh_litho_exp

### DRILL HOLE PETROPHYSICS INFORMATION

In [ ]:
sarig_dh_petrophys_exp = pd.read_csv(
    zipfile(os.path.join(data_loc, file_name),'r').open('SARIG_Data_Package/sarig_dh_petrophys_exp.csv','r'), 
    sep=',', encoding='latin1')
# sarig_dh_litho_exp['SAMREF_CNO'] = sarig_dh_litho_exp['SAMREF_CNO'].astype('Int64')
sarig_dh_petrophys_exp['LOGGING_DATE'] = pd.to_datetime(sarig_dh_petrophys_exp['LOGGING_DATE'])
sarig_dh_petrophys_exp.head(5)

In [ ]:
sarig_dh_petrophys_exp.dtypes

In [ ]:
sarig_dh_petrophys_exp[
    ['DRILLHOLE_NO', 'LOG_NO', 'DEPTH_FROM_M','DEPTH_TO_M', 
     'PETROPHYS_TYPE_CODE', 'VALUE', 'UNIT', 'SITE_NO']
]

Here, the INSTRUMENT and MEASURE_METHOD can be used to discuss the quality of data. 

In [ ]:
sarig_dh_petrophys_exp[['DRILLHOLE_NO', 'DEPTH_FROM_M', 'DEPTH_TO_M', 'PETROPHYS_TYPE_CODE', 'VALUE', 'UNIT']]

In [ ]:
sarig_dh_petrophys_exp['UNIT'].unique()

The VALUE column corresponds to different UNIT: ['SI', 'NOUNIT']. This affect the following feature engineering. Here, the UNIT will be expanded as columns ['SI', 'NOUNIT']. This leads to some NaN's but it will disappear when the PETROPHYS_TYPE_CODE converted as dummy variables. 

In [ ]:
sarig_dh_petrophys_exp['NOUNIT_VALUE'] = sarig_dh_petrophys_exp[sarig_dh_petrophys_exp['UNIT'] == 'NOUNIT']['VALUE']
sarig_dh_petrophys_exp['SI_VALUE'] = sarig_dh_petrophys_exp[sarig_dh_petrophys_exp['UNIT'] == 'SI']['VALUE']

In [ ]:
interested_dh_petrophys_exp =  sarig_dh_petrophys_exp[
    ['DRILLHOLE_NO', 'LOG_NO', 'DEPTH_FROM_M', 'DEPTH_TO_M', 
     'PETROPHYS_TYPE_CODE', 'NOUNIT_VALUE', 'SI_VALUE', 'SITE_NO']
].drop_duplicates()
del sarig_dh_petrophys_exp
interested_dh_petrophys_exp.sample(5)

In [ ]:
interested_dh_litho_exp.to_csv('./data/interested_dh_litho_exp.csv', sep=',', header='infer')

In [ ]:
interested_dh_litho_exp.shape, interested_dh_petrophys_exp.shape

### DRILL HOLE STRATIGRAPHIC INFORMATION

In [ ]:
sarig_dh_strat_exp = pd.read_csv(
    zipfile(os.path.join(data_loc, file_name),'r').open('SARIG_Data_Package/sarig_dh_strat_exp.csv','r'), 
    sep=',', encoding='latin1')
sarig_dh_strat_exp['LOGGING_DATE'] = pd.to_datetime(sarig_dh_strat_exp['LOGGING_DATE'])
sarig_dh_strat_exp.head(5)

In [ ]:
interest_dh_strat_exp = sarig_dh_strat_exp[[
  'DRILLHOLE_NO', 'DEPTH_FROM_M', 'DEPTH_TO_M', 'MAP_SYMBOL', 'GIS_CODE', 
  'MAJOR_LITHOLOGY_CODE', 'MINOR_LITHOLOGY_CODE', 'SITE_NO'  
]]


In [ ]:
del sarig_dh_strat_exp

### Export the extracted data

Here, the dataframes to be merged are saved and read again. This trick is to save memory usage as this codes is initially running on a PC with a RAM only 6GB. 

In [ ]:
# save the big dataframes 

interested_dh_litho_exp.to_csv('interested_dh_litho_exp.csv', 
                               sep=',', header='infer')
interested_dh_petrophys_exp.to_csv('interested_dh_petrophys_exp.csv',
                                  sep=',', header='infer')
interest_dh_strat_exp.to_csv('interest_dh_strat_exp.csv',
                                  sep=',', header='infer')

In [ ]:
# save the big dataframes 
import pandas as pd
import os
interested_dh_litho_exp = pd.read_csv('interested_dh_litho_exp.csv', 
                               sep=',', header='infer')
interested_dh_petrophys_exp = pd.read_csv('interested_dh_petrophys_exp.csv',
                                  sep=',', header='infer')
interest_dh_strat_exp= pd.read_csv('interest_dh_strat_exp.csv',
                                  sep=',', header='infer')
#remove the intermediate files
os.remove('interested_dh_litho_exp.csv')
os.remove('interested_dh_petrophys_exp.csv')
os.remove('interest_dh_strat_exp.csv')

In [ ]:
# # load the required SITE_NO from the csv file extracted from the rs_data.
path = '.\\data'
for directory in os.listdir(path):
    if os.path.isfile(os.path.join(path, directory)):
        pass
    else:
        new_path = os.path.join(path, directory)

        if os.path.exists(os.path.join(new_path, 'rs_chem_site_sample_num.csv')):
            rs_chem_site_sample_num = pd.read_csv(
                os.path.join(new_path, 'rs_chem_site_sample_num.csv'), 
                header='infer', 
                sep=',')['SITE_NO'].drop_duplicates()
            #print('read rs_chem_site_sample_num.csv successfully.')

            interested_dh_litho = interested_dh_litho_exp.merge(
                rs_chem_site_sample_num, how='inner', on='SITE_NO')

            interested_dh_petrophys = interested_dh_petrophys_exp.merge(
                rs_chem_site_sample_num, how='inner', on='SITE_NO')

            interest_dh_strat = interest_dh_strat_exp.merge(
                rs_chem_site_sample_num, how='inner', on='SITE_NO')

            extracted_dh_data = interested_dh_litho.merge(
                interested_dh_petrophys, 
                how='left', 
                on=['DRILLHOLE_NO', 'SITE_NO'], 
                suffixes=('', '_petro'))
            del interested_dh_litho
            del interested_dh_petrophys
            extracted_dh_data = extracted_dh_data.merge(
                interest_dh_strat,
                how='left', 
                on=['DRILLHOLE_NO', 'SITE_NO'], 
                suffixes=('', '_strat'))
            del interest_dh_strat
            extracted_dh_data.to_csv(os.path.join(new_path,'extracted_dh_data.csv'), sep=',', header='infer') 
        else:
            pass